In [0]:
pip install sentencepiece

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration, pipeline
model_name = "facebook/m2m100_418M"
base_data_path = "/Users/Kenneth.Johnson/Documents/ml/huggingface/m2m100_418M/"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
nlp = pipeline("translation", model=model, tokenizer=tokenizer)

In [0]:
nlp.save_pretrained(base_data_path)

In [0]:
from logging import shutdown
import mlflow
import json
import pandas as pd
import shutil
import os

code_root_path = "/Workspace/Repos/kenneth.johnson@databricks.com/translation-demo/"

f = open(os.path.join(code_root_path ,"conda_envs/conda_translation.json"))
conda_env = json.load(f)
f.close()

def test_conda_env():
    assert isinstance(conda_env, dict)

with mlflow.start_run(run_name="translation_m2m100") as run:
    mlflow.pyfunc.log_model(artifact_path="translation", 
        loader_module="translation_transformer", 
        conda_env=conda_env, code_path=[os.path.join(code_root_path,"pyfunc_modules/translation_transformer.py")], 
        data_path=base_data_path.replace("dbfs:", "/dbfs")
    )

translation_loaded = mlflow.pyfunc.load_model(f"runs:/{run.info.run_id}/translation")

def test_model_load():
    assert(isinstance(translation_loaded, mlflow.pyfunc.PyFuncModel))

test_df = pd.DataFrame({"id":[1,2,3], "content": ["Abraham Lincoln cut down a cherry tree", "Florida has nice beaches", "Elon Musk owns Tesla"]})

#mlflow.pyfunc.PythonModel enforces a one argument predict function so we use a tuple to send in our src and target languages
param_dict = {'src_lang': 'en', 'target_lang': 'pt', 'batch_size': 1}
model_input = ([test_df, param_dict])
results = translation_loaded.predict(model_input)

def test_model_inference():
    assert(results.columns.values.tolist() == ['id', 'content', 'translation'] and results.shape[0]==3)

pd.set_option('display.max_colwidth', None)


# clean up
#mlflow.delete_run(run.info.run_id)
# used in local environment tests. Not necessary when running in Databricks
#shutil.rmtree(f"mlruns/0/{run.info.run_id}/", ignore_errors=True)

/databricks/python/lib/python3.9/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
['Abraham Lincoln cortou uma árvore de cerveja', 'A Flórida tem belas praias', 'Elon Musk é proprietário da Tesla']


In [0]:
test_conda_env()
test_model_load() 
test_model_inference() 
results 

,id,content,translation
0,1,Abraham Lincoln cut down a cherry tree,Abraham Lincoln cortou uma árvore de cerveja
1,2,Florida has nice beaches,A Flórida tem belas praias
2,3,Elon Musk owns Tesla,Elon Musk é proprietário da Tesla


In [0]:
# clean up
#mlflow.delete_run(run.info.run_id)

In [0]:
result = test_model_load()
result

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-649139525163380> in <cell line: 1>()
----> 1 result = test_model_load()
      2 result

NameError: name 'test_model_load' is not defined